재무제표 데이터 수집 파일입니다. 

우선 코스피 상위 100개 종목의 명칭을 크롤링한 후 각 기업의 재무제표를 가져오는 과정입니다.

1. 코스피 상위 100개 크롤링

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

data = []

# 1~2 페이지 순회
for page in [1, 2]:
    url = f"https://stock.mk.co.kr/domestic/ranking/market_cap?page={page}"
    res = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(res.text, "html.parser")

    table = soup.select_one("table")
    rows = table.select("tr")[1:]  # 첫 행은 헤더

    for row in rows:
        cols = [td.get_text(strip=True) for td in row.find_all("td")]
        if not cols:
            continue
        data.append({
            "종목명": cols[0],
            "현재가": cols[1],
            "등락률": cols[2],
            "시가총액": cols[4]
        })

df = pd.DataFrame(data)
df.tail()




C:\Users\mcdor\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,종목명,현재가,등락률,시가총액
95,삼성E&A,"23,900",700,"4,684,400"
96,LG디스플레이,"9,300",220,"4,650,000"
97,현대제철,"34,750",800,"4,637,241"
98,KODEX 미국S&P500,"19,505",110,"4,588,551"
99,오리온,"116,000","2,500","4,586,191"


In [2]:
kospi100 = df.iloc[:, 0].tolist()
kospi100


['삼성전자',
 'SK하이닉스',
 '삼성바이오로직스',
 'LG에너지솔루션',
 '현대차',
 '삼성전자우',
 'KB금융',
 '셀트리온',
 'NAVER',
 '기아',
 '두산에너빌리티',
 '한화에어로스페이스',
 'HD현대중공업',
 '신한지주',
 '삼성물산',
 '현대모비스',
 '카카오',
 'POSCO홀딩스',
 'HMM',
 '삼성생명',
 '하나금융지주',
 '한국전력',
 '한화오션',
 'HD한국조선해양',
 'SK스퀘어',
 '메리츠금융지주',
 '삼성화재',
 '현대로템',
 'LG화학',
 'SK이노베이션',
 '크래프톤',
 '우리금융지주',
 '고려아연',
 'KT&G',
 'HD현대일렉트릭',
 '기업은행',
 '삼성SDI',
 'KT',
 'SK',
 '카카오뱅크',
 '삼성중공업',
 '삼성에스디에스',
 'LG전자',
 'LG',
 'SK텔레콤',
 '하이브',
 '미래에셋증권',
 'LIG넥스원',
 '카카오페이',
 '삼성전기',
 '포스코퓨처엠',
 '현대글로비스',
 '한화시스템',
 'HD현대',
 '삼양식품',
 '두산',
 '포스코인터내셔널',
 '한미반도체',
 'DB손해보험',
 '대한항공',
 '유한양행',
 'KODEX CD금리액티브(합성)',
 'HD현대마린솔루션',
 'TIGER 미국S&P500',
 '효성중공업',
 '한국항공우주',
 '한진칼',
 '현대건설',
 'LS ELECTRIC',
 '아모레퍼시픽',
 'LG씨엔에스',
 '한국금융지주',
 'SK바이오팜',
 'S-Oil',
 '코웨이',
 'HD현대미포',
 'KODEX 200',
 'KODEX 머니마켓액티브',
 'NH투자증권',
 'LG유플러스',
 '한화',
 '한화솔루션',
 '삼성증권',
 '삼성카드',
 '현대차2우B',
 'LS',
 '맥쿼리인프라',
 '한국타이어앤테크놀로지',
 '키움증권',
 '넷마블',
 '에이피알',
 '두산밥캣',
 'TIGER 미국나스닥100',
 'LG생활건강',
 'TIGER CD금리투

2. 재무제표 크롤링

(1) 삼성전자 예시 

In [3]:
#!pip install opendartreader #필요시 설치
import OpenDartReader 

In [4]:
api_key ='699356c21478b2299c8a02f5ace80239bdb2f344'

In [5]:
dart=OpenDartReader(api_key)
dart

In [6]:
dart_2025=dart.finstate("삼성전자",2024)
dart_2025.columns

Index(['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'stock_code',
       'fs_div', 'fs_nm', 'sj_div', 'sj_nm', 'account_nm', 'thstrm_nm',
       'thstrm_dt', 'thstrm_amount', 'frmtrm_nm', 'frmtrm_dt', 'frmtrm_amount',
       'bfefrmtrm_nm', 'bfefrmtrm_dt', 'bfefrmtrm_amount', 'ord', 'currency'],
      dtype='object')

In [7]:
#컬럼이 영어라 한국어로 바꿉니다
column_map = {
    'rcept_no': '접수번호',
    'reprt_code': '보고서코드',
    'bsns_year': '사업연도',
    'corp_code': '회사코드',
    'stock_code': '종목코드',
    'fs_div': '연결/별도 구분',
    'fs_nm': '재무제표명',
    'sj_div': '재무제표종류코드',
    'sj_nm': '재무제표종류',
    'account_nm': '계정명',
    'thstrm_nm': '당기명칭',
    'thstrm_dt': '당기종료일',
    'thstrm_amount': '당기금액',
    'frmtrm_nm': '전기명칭',
    'frmtrm_dt': '전기종료일',
    'frmtrm_amount': '전기금액',
    'bfefrmtrm_nm': '전전기명칭',
    'bfefrmtrm_dt': '전전기종료일',
    'bfefrmtrm_amount': '전전기금액',
    'ord': '출력순서',
    'currency': '통화단위'
}


In [8]:
dart_2025.rename(columns=column_map, inplace=True)
dart_2025.tail()


,접수번호,보고서코드,사업연도,회사코드,종목코드,연결/별도 구분,재무제표명,재무제표종류코드,재무제표종류,계정명,...,당기종료일,당기금액,전기명칭,전기종료일,전기금액,전전기명칭,전전기종료일,전전기금액,출력순서,통화단위
25,20250311001085,11011,2024,00126380,005930,OFS,재무제표,IS,손익계산서,영업이익,...,2024.01.01 ~ 2024.12.31,"12,361,034,000,000",제 55 기,2023.01.01 ~ 2023.12.31,"-11,526,297,000,000",제 54 기,2022.01.01 ~ 2022.12.31,"25,319,329,000,000",26,KRW
26,20250311001085,11011,2024,00126380,005930,OFS,재무제표,IS,손익계산서,법인세차감전 순이익,...,2024.01.01 ~ 2024.12.31,"21,749,578,000,000",제 55 기,2023.01.01 ~ 2023.12.31,"17,531,500,000,000",제 54 기,2022.01.01 ~ 2022.12.31,"29,691,920,000,000",28,KRW
27,20250311001085,11011,2024,00126380,005930,OFS,재무제표,IS,손익계산서,당기순이익,...,2024.01.01 ~ 2024.12.31,"23,582,565,000,000",제 55 기,2023.01.01 ~ 2023.12.31,"25,397,099,000,000",제 54 기,2022.01.01 ~ 2022.12.31,"25,418,778,000,000",30,KRW
28,20250311001085,11011,2024,00126380,005930,OFS,재무제표,IS,손익계산서,당기순이익(손실),...,2024.01.01 ~ 2024.12.31,"23,582,565,000,000",제 55 기,2023.01.01 ~ 2023.12.31,"25,397,099,000,000",제 54 기,2022.01.01 ~ 2022.12.31,"25,418,778,000,000",62,KRW
29,20250311001085,11011,2024,00126380,005930,OFS,재무제표,IS,손익계산서,총포괄손익,...,2024.01.01 ~ 2024.12.31,"23,230,095,000,000",제 55 기,2023.01.01 ~ 2023.12.31,"25,181,020,000,000",제 54 기,2022.01.01 ~ 2022.12.31,"26,031,896,000,000",64,KRW


-> 여기서 계정명 컬럼과 당기금액 컬럼만 필요

In [9]:
dart_2025_main =dart_2025[['계정명','당기금액']] #컬럼중 계정명과 당기금액이 중요
dart_2025_main

,계정명,당기금액
0,유동자산,"227,062,266,000,000"
1,비유동자산,"287,469,682,000,000"
2,자산총계,"514,531,948,000,000"
3,유동부채,"93,326,299,000,000"
4,비유동부채,"19,013,579,000,000"
5,부채총계,"112,339,878,000,000"
6,자본금,"897,514,000,000"
7,이익잉여금,"370,513,188,000,000"
8,자본총계,"402,192,070,000,000"
9,매출액,"300,870,903,000,000"


(2) kospi 시총 상위 100개 리스트 재무제표 크롤링

In [15]:
import pandas as pd
from tqdm import tqdm
import OpenDartReader
api_key = '699356c21478b2299c8a02f5ace80239bdb2f344'
dart = OpenDartReader(api_key)
main_df = pd.DataFrame()

for stock in tqdm(kospi100):
    try:
        df = dart.finstate(corp=stock, bsns_year=2025, reprt_code='11013')  #2025 1분기 보고서
        
        if df is None or df.empty:
            print(f"{stock} 재무제표 없음")
            continue
        
        # 연결재무제표만 필터
        df = df[df['fs_nm'] == '연결재무제표']
        
        # 'account_nm'과 당기금액('thstrm_amount')만 가져옴
        df_main = df[['account_nm', 'thstrm_amount']].copy()
        df_main.columns = ['계정명', stock]
        
        if main_df.empty:
            main_df = df_main
        else:
            main_df = pd.merge(main_df, df_main, on='계정명', how='outer')

    except Exception as e:
        print(f"{stock} 처리 중 오류: {e}")

main_df = main_df.drop_duplicates(subset='계정명').set_index('계정명')

main_df

  4%|▍         | 4/100 [00:04<01:54,  1.19s/it]

현대차 처리 중 오류: argument of type 'NoneType' is not iterable
삼성전자우 처리 중 오류: argument of type 'NoneType' is not iterable


  8%|▊         | 8/100 [00:07<01:10,  1.31it/s]

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

셀트리온 재무제표 없음


 10%|█         | 10/100 [00:09<01:21,  1.11it/s]

...

 98%|█████████▊| 98/100 [01:38<00:01,  1.12it/s]

KODEX 미국S&P500 처리 중 오류: argument of type 'NoneType' is not iterable


100%|██████████| 100/100 [01:40<00:00,  1.00s/it]

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

오리온 재무제표 없음


,삼성전자,SK하이닉스,삼성바이오로직스,LG에너지솔루션,KB금융,NAVER,두산에너빌리티,한화에어로스페이스,HD현대중공업,신한지주,...,삼성카드,LS,한국타이어앤테크놀로지,키움증권,에이피알,두산밥캣,LG생활건강,삼성E&A,LG디스플레이,현대제철
계정명,,,,,,,,,,,,,,,,,,,,,
기타포괄손익-공정가치측정금융자산,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"547,652,464,683",NaN,NaN,NaN,NaN,NaN,NaN
당기손익-공정가치측정금융자산,NaN,NaN,NaN,NaN,"84,218,327,000,000",NaN,NaN,NaN,NaN,"73,491,188,000,000",...,NaN,NaN,NaN,"31,556,374,965,327",NaN,NaN,NaN,NaN,NaN,NaN
당기순이익,"8,222,878,000,000","8,108,195,000,000","375,553,550,929","226,573,000,000","1,699,122,000,000","423,661,329,790","-21,157,000,000","209,348,668,000","284,230,314,000","1,517,046,000,000",...,"184,440,330,982","141,111,000,000","312,207,573,348","235,599,580,315","49,941,244,107","78,042,000","103,459,072,938","157,212,353,869","-237,032,000,000","-54,393,174,968"
당기순이익(손실),"8,222,878,000,000","8,108,195,000,000","375,553,550,929","226,573,000,000","1,699,122,000,000","423,661,329,790","-21,157,000,000","209,348,668,000","284,230,314,000","1,517,046,000,000",...,"184,440,330,982","141,111,000,000","312,207,573,348","235,599,580,315","49,941,244,107","78,042,000","103,459,072,938","157,212,353,869","-237,032,000,000","-54,393,174,968"
매출액,"79,140,503,000,000","17,639,141,000,000","1,298,272,949,854","6,264,995,000,000",NaN,"2,786,783,351,907","3,748,574,000,000","5,484,220,608,000","3,822,515,741,000",NaN,...,"1,067,407,617,453","6,913,573,000,000","4,963,667,180,882","3,659,953,408,737","266,032,725,955","1,444,356,000","1,697,877,264,205","2,097,955,167,196","6,065,298,000,000","5,563,465,036,922"
법인세차감전 순이익,"9,151,576,000,000","9,299,229,000,000","499,386,550,907","364,506,000,000","2,306,606,000,000","580,108,521,984","30,207,000,000","261,035,970,000","376,264,685,000","2,005,558,000,000",...,"246,691,709,936","194,888,000,000","393,380,555,694","326,088,236,418","59,178,654,502","116,892,000","142,062,812,004","204,633,437,489","-148,495,000,000","-66,975,696,767"
보험계약부채,NaN,NaN,NaN,NaN,"59,243,225,000,000",NaN,NaN,NaN,NaN,"52,872,801,000,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
보험계약자산,NaN,NaN,NaN,NaN,"301,214,000,000",NaN,NaN,NaN,NaN,"2,312,000,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
부채총계,"109,762,479,000,000","42,546,717,000,000","6,156,868,520,968","31,028,433,000,000","710,192,373,000,000","11,284,404,608,985","15,153,864,000,000","33,698,164,412,000","15,098,481,877,000","686,874,741,000,000",...,"21,934,393,696,232","14,508,413,000,000","12,033,741,941,034","54,730,763,471,594","235,668,245,170","3,689,199,000","1,821,819,436,913","6,042,997,589,177","24,153,761,000,000","15,439,622,878,862"


결측치가 있는 것은 기업별로 재무제표가 다르기 때문입니다.<br>
그리고 컬럼이 100개가 아닌 것은 삼성전자, 삼성화학과 같이 같은 부류의 회사들은 시총이가장 큰 회사의 재무제표만 데이터가 나오기 때문입니다.

또한 코드를 보면 2025 1분기 보고서로 진행하였으며, 1분기 보고서를 아직 제출 안한 기업들은 데이터를 가져올 수 없습니다.